In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####  -Done
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name  -Done

username = "aacuser"
password = "SNHU1234"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo -Done
image_filename = 'logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date -Done
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

# Creates a div for the logo on the left and the header and my
# unique identifier on the right
app.layout = html.Div([
    html.Div(
        style= {'display':'flex','alignItems':'center','marginBottom':'20px'},
        children=[
            html.A(
                # Link to SNHU as requested in documentation
                href="https://www.snhu.edu",
                target='_blank',
                children=html.Img(
                    src='data:logo/png;base64,{}'.format(encoded_image.decode()),
                    style={'height':'100px','margin-right':'20px'}
                )
            ),
            html.Div([
                html.H2("CS-340 Dashboard"),
                html.P("Alex Hitchens - Dashboard Project")
            ])
        ]
    ),
  
    html.Hr(),
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down,etc. -Done    
# Filter options are radio buttons to differenct rescue type
    dcc.RadioItems(
        id = 'filter-type',
        options=[
            {'label':'Water Rescue', 'value':'Water'},
            {'label':'Mountain Rescue', 'value':'Mountain'},
            {'label':'Disaster Rescue', 'value':'Disaster'},
            {'label':'Reset', 'value':'All'},
        ],
        value = 'All',
        labelStyle={'display': 'inline-block','margin-right':'10px'}


    ),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here -Done
    row_selectable = "single",                       
    selected_rows=[0],
                         
    # Options for client to sort, as well as showing only 10 animals per page.
    sort_action = 'native',
    page_action = "native",
    page_current= 0,
    page_size = 10,
                        ),

    html.Br(),
    html.Hr(),
    
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

### Data Table ###
# This callback / function will update the graph to show a percentage break down or
# the animal breeds in the data table, I truncated everything below 1% so as to make
# the data useful and not overwhelming.

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])

def update_graphs(viewData):
    
    # Check if there is no data
    if viewData is None or len(viewData)==0:
        return html.Div("No Data to Display.")
    
    #set the data frame
    dff = pd.DataFrame.from_dict(viewData)
    
    # Just in case the dataframe doesn't have anything with a breed
    if 'breed' not in dff.columns:
        return html.Div("Breed Data not Available.")
    
    # Truncate anything that isn't a dog
    dff = dff[dff['animal_type']=='Dog']
    
    # if there is no dogs return that fact
    if dff.empty:
        return html.Div("No dog data available.")
    
    # this truncates anything that is, or is below 1% of the breed population
    # This prevents  a bunch of .01% showing up.
    breed_counts = dff['breed'].value_counts(normalize=True) * 100
    top_breeds = breed_counts[breed_counts >1].index
    filtered_df = dff[dff['breed'].isin(top_breeds)]
    
    # set the pie chart up and return it
    fig = px.pie(filtered_df, names='breed', title='Rescue Dog Breed Distribution (>1%)')
    return dcc.Graph(figure=fig)
    
### Filters ###
# This callback will take the datasheet and filter it to show only
# cats or dogs based on which button is clicked.
@app.callback(
    Output('datatable-id',"data"),
    [Input('filter-type','value')])

def filter(rescue_type):
    
    # this sets the queary to the requested types based on the purpose
    if rescue_type =='Water':
        query = {
            "breed":{
                "$in":["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {
                "$gte": 26,
                "$lte": 156}        
        }
       
    elif rescue_type == 'Mountain':
        query = {
            "breed":{
                "$in":["German Shepherd", "Alaskan Malamute", "Old English Sheepdog",
                      "Siberian Husky","Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {
                "$gte": 26,
                "$lte": 156}        
        }
    elif rescue_type == 'Disaster':
        query = {
            "breed":{
                "$in":["Doberman Pinscher", "German Shepherd", "Golden Retriever",
                       "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {
                "$gte": 20,
                "$lte": 300}        
        }
    else:
        query = {}
            
    # query mongodb and convert list into dataframe df     
    df = pd.DataFrame.from_records(db.read(query))    
    
    # drop the _id field
    if '_id' in df.columns:
        df.drop(columns=['_id'],inplace=True)
    
    # return the formatted dataframe
    return df.to_dict('records')

### Reset Selection ###
# This resets the selection everytime the filter changes to avoid selecting an out of bounds
# row and then filtering to something that doesn't have that many rows.
@app.callback(
    Output('datatable-id',"selected_rows"),
    [Input('filter-type','value')])

def reset_selected_rows(rescue_type):
    return[0]

### Highlight Row ###
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if not selected_columns:
        return []
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

### Map ###
# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    #Check if no data present
    if viewData is None or index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    
    row = index[0]
    
    # Returns formatted map
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]

app.run_server(debug=True)


Dash app running on http://127.0.0.1:27312/
